In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import json
import datetime
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from GoogleCalendarApi import GoogleCalendar

In [3]:
GCA = GoogleCalendar()
GCA.login()

login success


In [ ]:
service = GCA.i_want_try_some_thing()

In [5]:
GCA.calendar_list()

[{'kind': 'calendar#calendarListEntry',
  'etag': '"1619782656629000"',
  'id': 'serverqaq88@gmail.com',
  'summary': 'serverqaq88@gmail.com',
  'timeZone': 'Asia/Taipei',
  'colorId': '14',
  'backgroundColor': '#9fe1e7',
  'foregroundColor': '#000000',
  'accessRole': 'owner',
  'defaultReminders': [{'method': 'popup', 'minutes': 30}],
  'notificationSettings': {'notifications': [{'type': 'eventCreation',
     'method': 'email'},
    {'type': 'eventChange', 'method': 'email'},
    {'type': 'eventCancellation', 'method': 'email'},
    {'type': 'eventResponse', 'method': 'email'}]},
  'primary': True,
  'conferenceProperties': {'allowedConferenceSolutionTypes': ['hangoutsMeet']}},
 {'kind': 'calendar#calendarListEntry',
  'etag': '"1619786950173000"',
  'id': '0rlr2avhoavmv3j695rls45528@group.calendar.google.com',
  'summary': 'huashan1914',
  'description': 'huashan1914',
  'timeZone': 'Asia/Taipei',
  'colorId': '8',
  'backgroundColor': '#16a765',
  'foregroundColor': '#000000',
  '

In [6]:
huashan1914ID = "0rlr2avhoavmv3j695rls45528@group.calendar.google.com"
GCA.set_calendarid(huashan1914ID)

In [7]:
GCA.events_list()

[{'kind': 'calendar#event',
  'etag': '"3239576595266000"',
  'id': 'n8336s5869211goak4tis0uprk',
  'status': 'confirmed',
  'htmlLink': 'https://www.google.com/calendar/event?eid=bjgzMzZzNTg2OTIxMWdvYWs0dGlzMHVwcmsgMHJscjJhdmhvYXZtdjNqNjk1cmxzNDU1MjhAZw',
  'created': '2021-04-30T13:11:37.000Z',
  'updated': '2021-04-30T13:11:37.633Z',
  'summary': '島嶼採集課《拓印植物圖鑑》',
  'description': '島嶼採集課《拓印植物圖鑑》<br>展演活動<br>2021-05-01 14:00:00<br>2021-05-01 16:30:00<br><a href="https://www.huashan1914.com/w/huashan1914/exhibition_21031014532645113" __is_owner="true">https://www.huashan1914.com/w/huashan1914/exhibition_21031014532645113</a><br>===== 活動情形皆已官網為主 =====<br>',
  'location': '100 台北市中正區八德路一段1號',
  'creator': {'email': 'serverqaq88@gmail.com'},
  'organizer': {'email': '0rlr2avhoavmv3j695rls45528@group.calendar.google.com',
   'displayName': 'huashan1914',
   'self': True},
  'start': {'dateTime': '2021-05-02T14:30:00+08:00',
   'timeZone': 'Asia/Taipei'},
  'end': {'dateTime': '2021-05-02T14

In [ ]:
# GCA.delete_all()

In [ ]:
CheckPageInfo = True
CheckPageNum = 1
Page = []
while CheckPageInfo:
    req1 = requests.get(f'https://www.huashan1914.com/w/huashan1914/exhibition?index={CheckPageNum}')
    soup1 = bs(req1.text, 'html5lib')
    findAllList = soup1.find('ul', {'id':'event-ul'}).findAll('li')
    if findAllList:
        for i in findAllList:
            activityName = i.find("div", {"class":"card-text-name"}).text
            activityDate = i.find("div", {"class":"event-date"}).text.strip()
            activityTime = "1:00 AM - 11:59 PM" if i.find("div", {"class":"event-time"}) is None else i.find("div", {"class":"event-time"}).text.strip()
            activityDateCut = activityDate.split(' - ')
            activityTimeCut = activityTime.split(' - ')
            
            activityTimeStart = datetime.datetime.strptime(activityTimeCut[0], "%I:%M %p").strftime("%H:%M:%S")
            activityTimeEnd = datetime.datetime.strptime(activityTimeCut[1], "%I:%M %p").strftime("%H:%M:%S")
            
            if len(activityDateCut) == 2:
                if len(activityDateCut[1]) > 6:
                    # 跨年分活動
                    activityDateStart, activityDateEnd = activityDateCut[0], activityDateCut[1]
                else:
                    # 當一年活動
                    activityDateStart, activityDateEnd = activityDateCut[0], activityDateCut[0][0:5] + activityDateCut[1]
            else:
                # 當天活動
                activityDateStart, activityDateEnd = activityDateCut[0], activityDateCut[0]

            activityUrl = 'https://www.huashan1914.com' + i.find("a")['href']
            activityType = "|".join([c.text for c in i.find("div", {'class':'event-list-type'}).findAll('span')])
            datetime.datetime.strptime("11:59 PM", "%I:%M %p").strftime("%H:%M")
            Page.append((activityName, activityDateStart, activityDateEnd, activityUrl, activityType, activityTimeStart, activityTimeEnd))
        CheckPageNum+=1
#         break
    else:
        CheckPageInfo = False

In [ ]:
DF = pd.DataFrame(Page, columns=['Title', 'StartDate', 'EndDate', 'Url', 'Description', "StartTime", "EndTime"])

In [ ]:
DF['StartDate'] = pd.to_datetime(DF['StartDate'], format='%Y-%m-%d').astype('str')
DF['EndDate'] = pd.to_datetime(DF['EndDate'], format='%Y-%m-%d').astype('str')

In [ ]:
inputttt = DF.to_dict(orient="records")

In [ ]:
location = '100 台北市中正區八德路一段1號'
for i in inputttt:
    event = {
      'summary': i["Title"],
      'location': location,
      'description': (f'{i["Title"]}<br>'
                      f'{i["Description"]}<br>'
                      f'{i["StartDate"]} {i["StartTime"]}<br>'
                      f'{i["EndDate"]} {i["EndTime"]}<br>'
                      f'<a href="{i["Url"]}" __is_owner="true">{i["Url"]}</a><br>'
                      "===== 活動情形皆已官網為主 =====<br>"
                     ),
      'start': {
        'dateTime': f'{i["StartDate"]}T{i["StartTime"]}-{i["EndTime"]}',
        'timeZone': 'Asia/Taipei',
      },
      'end': {
        'dateTime': f'{i["EndDate"]}T{i["StartTime"]}-{i["EndTime"]}',
        'timeZone': 'Asia/Taipei',
      },
    }
    GCA.insert(event)
    